In [1]:
import  pandas as pd
import  numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import xlrd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import openpyxl
import xgboost as xgb
from sklearn.feature_selection import RFE
from concurrent.futures import ThreadPoolExecutor
import queue
import warnings
import joblib
from tqdm import tqdm
%run myfun.ipynb
import os
import glob

warnings.filterwarnings("ignore")
clf = joblib.load("D:\\RFModel\\rf_model.m")
selector = joblib.load('D:\\RFModel\\rf_model_selector.pkl')

In [34]:
from tqdm import tqdm
feature_start = 10
feature_num = 50

result_path = r"\Fraction_11"
path_directory = os.path.dirname(result_path)
path = os.path.join(path_directory, "out_all_prsm.csv")


model_path = 'random_forest_model.pkl'
selector_path = 'selector.pkl'


def read_single_csv(input_path):
    total_rows = sum(1 for _ in open(input_path, encoding='utf-8')) - 1
    chunks = pd.read_csv(input_path, chunksize=10000)
    df_list = []
    for chunk in tqdm(chunks, total=total_rows // 10000 + 1, desc="Reading CSV"):
        df_list.append(chunk)
    return pd.concat(df_list, ignore_index=True)

print("Reading and preprocessing data...")
data = read_single_csv(path)


mean_fill_cols = ['feature21', 'feature22', 'feature23']
data[mean_fill_cols] = data[mean_fill_cols].fillna(data[mean_fill_cols].mean())

data[['feature46', 'feature47', 'feature48']] = data[['feature46', 'feature47', 'feature48']].fillna(0)
data['feature48'] = data['feature48'].replace([np.inf, -np.inf], 0)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)



print("Applying feature selection and predicting...")
raw_data = data.copy()


numerical_features = data.iloc[:, feature_start:feature_start + feature_num].astype(float)


test_feature_selected = selector.transform(numerical_features.astype(float))


probabilities = clf.predict_proba(test_feature_selected)
class_1_probs = probabilities[:, 1]
raw_data['score'] = class_1_probs


for i in tqdm(range(len(data)), desc="Predicting scores"):
    raw_data['score'].iloc[i] = class_1_probs[i]


raw_data.sort_values(by=[raw_data.columns[0], 'score', raw_data.columns[9]], ascending=[True, False, False], inplace=True)
raw_data.to_csv(result_path + '_candi_output.csv', index=False)


grouped_best = (
    raw_data.sort_values(by=['score', 'match fragment'], ascending=[False, False])
            .groupby(raw_data.columns[0], as_index=False)
            .first()
)


grouped_best.sort_values(by=['score', grouped_best.columns[9]], ascending=[False, False], inplace=True)
grouped_best.to_csv(result_path + '_output.csv', index=False)

print("Prediction completed and results saved.")

Reading and preprocessing data...


Reading CSV: 100%|███████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.18it/s]
[Parallel(n_jobs=30)]: Using backend ThreadingBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 140 tasks      | elapsed:    0.0s


Applying feature selection and predicting...


[Parallel(n_jobs=30)]: Done 390 tasks      | elapsed:    0.1s
[Parallel(n_jobs=30)]: Done 740 tasks      | elapsed:    0.2s
[Parallel(n_jobs=30)]: Done 1000 out of 1000 | elapsed:    0.3s finished
Predicting scores: 100%|███████████████████████████████████████████████████████| 12502/12502 [00:02<00:00, 4675.86it/s]


Prediction completed and results saved.


In [ ]:
mean_fill_cols = ['feature21', 'feature22', 'feature23']
data[mean_fill_cols] = data[mean_fill_cols].fillna(data[mean_fill_cols].mean())

data[['feature46', 'feature47', 'feature48']] = data[['feature46', 'feature47', 'feature48']].fillna(0)
data['feature48'] = data['feature48'].replace([np.inf, -np.inf], 0)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

raw_data = data.copy()

numerical_features = data.iloc[:, feature_start:feature_start + feature_num].astype(float)

test_feature_selected = selector.transform(numerical_features.astype(float))

probabilities = clf.predict_proba(test_feature_selected)
class_1_probs = probabilities[:, 1]
raw_data['score'] = class_1_probs

raw_data.sort_values(by=[raw_data.columns[0], 'score', raw_data.columns[9]], ascending=[True, False, False], inplace=True)
raw_data.to_csv(result_path + '_candi_output.csv', index=False)


grouped_best = (
    raw_data.sort_values(by=['score', 'match fragment'], ascending=[False, False])
            .groupby(raw_data.columns[0], as_index=False)
            .first()
)


grouped_best.sort_values(by=['score', grouped_best.columns[9]], ascending=[False, False], inplace=True)
grouped_best.to_csv(result_path + '_output.csv', index=False)
